## Hydrology 


### Contribution to discharge

- discharge (obs vs sim)
- base flow 
- lateral flow 
- runoff
- drainage

In [1]:
import datetime
import os

import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import numpy as np

from obs_data import *

print(os.getcwd())

2016
D:\Documents\these_pablo\Models\BEACH2016\Analysis


In [2]:
PC = True
analysis = True
short_sim = True
version = "model_v12"
folder = "1"

timestamp = 'julian'

In [3]:
if analysis:
    dot = ''
else:
    dot = '..\\'
    
if PC:
    back = "\\"  
    sim_path = dot + version + back  
else:
    back = "/"
    sim_path = version + back
    
print(sim_path)

model_v12\


## Observed

In [4]:
q_obs = getWaterData(tss=True)
q_obs.loc[q_obs.Qm3 < 0,'Qm3'] = np.nan

In [5]:
if short_sim:
    s_obs = 166 - 1
    e_obs = 300
else:
    # Julian days
    s_obs = 0
    e_obs = 360

# Calendar days
if timestamp == 'julian':
    # Julian days
    x = q_obs.Jdays.tolist()[s_obs:e_obs]
else:
    # Calendar days
    dates = getTime(sep=";")
    x = dates.Date.tolist()[s_obs:e_obs]
    # dates = getTime(sep=";")
    # x = dates.Date.tolist()[s:e]

    

In [6]:
# Observed 'Q (m3/d)'
# q_obs[s_obs:(s_obs+20)]
qObs = q_obs.Qm3.tolist()[s_obs:e_obs]

trace = go.Scatter(
        x = x,
        y = qObs,
        mode = 'lines',
        name = 'Q (m3/d)',
        marker=dict(
            color= 'green')
)

## Simulated

In [7]:
tss = ["tot_accVol_m3", # Q sim
       "o_cellLatflow_m3",  # Outlet latflow (m3)
       "accRunoff_m3",  # Runoff (m3)
       "o_accDrain_m3",  # Drainage (m3)
       "accPercol_Bsmt_m3", # Percolation Basement (m3)
      'accBaseflow_m3' 
      ]  # Baseflow, linear reservoir (m3)

names = ['Q Sim.', 'Latflow Sim.', 'Runoff Sim.', 'Drainage Sim.', 
         'Deep Percol Sim.', 
         'Baseflow Sim.'
        ]

palette = ['blue', 'purple', 'rgb(178,223,138)', '#F44336', '#3F51B5', 'orange']

col = "sim"

data = []
data.append(trace)  # Append observed Q.
for i in range(len(tss)):  # Append simulated components
    filename = "resW_" + tss[i] + ".tss"
    sim_pd = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', col],
                            header=None)
    
    sim = sim_pd.sim.tolist()
    trace = go.Scatter(
        x = x,
        y = sim,
        mode = 'lines',
        name = names[i],
        line = dict(
            color = palette[i],
            width = 3,
            dash = 'dot')
    )
    data.append(trace)


layout = go.Layout(
    title='Discharge Components',
    xaxis=dict(
        title='',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title=r'$Q (m^3/d)$',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

In [8]:
figname = "Qcomponents.png"

%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data, layout=layout)
    py.image.save_as(fig, filename=figname)
    # py.plot(fig, filename=figname) # Opens in web
else:
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


### Q Components $(m^3/d)$

Follow this link to interact with the data:

https://plot.ly/~DayTimeChunks/15/


In [9]:
from IPython.display import Image
if online:
    i = Image(figname)
    display(i)

## Saturation extent

In [10]:
props = ['z0_thetaPropSat', 'z1_thetaPropSat', 'z2_thetaPropSat', 'z3_thetaPropSat', 'Bsmt_thetaPropSat']
names_p = [r'$\theta_{z0}$', r'$\theta_{z1}$', r'$\theta_{z2}$', r'$\theta_{z3}$', r'$\theta_{Bsmt}$']

data_p = []
for i in range(len(props)):  # Append simulated components
    filename = "resW_" + props[i] + ".tss"
    sim_pd = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', 'sim'],
                            header=None)
    
    sim = sim_pd.sim.tolist()
    trace = go.Scatter(
        x = x,
        y = sim,
        mode = 'lines',
        name = names_p[i],
        line = dict(
            color = palette[i],
            width = 3,
            dash = 'dot')
    )
    data_p.append(trace)

figname = "ThetaProp.png"
%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data_p)
    py.image.save_as(fig, filename=figname)
    # py.plot(fig, filename=figname) # Opens in web
else:
    fig = go.Figure(data=data_p)
    iplot(fig)

## Water balance z3

In [11]:
ts = ['accBAL_Bsmt_m3', 'accInfil_Bsmt_m3', 'accLF_Bsmt_m3', 'accETP_Bsmt_m3', 
      # 'accBaseflow_m3'
     ]
data_z3 = []
for i in range(len(ts)):  # Append simulated components
    filename = "resW_" + ts[i] + ".tss"
    sim_pd = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', 'sim'],
                            header=None)
    
    sim = sim_pd.sim.tolist()
    trace = go.Scatter(
        x = x,
        y = sim,
        mode = 'lines',
        name = ts[i],
        line = dict(
            color = palette[i],
            width = 3,
            dash = 'dot')
    )
    data_z3.append(trace)

figname = "BalanceWaterZ3.png"
%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data_z3)
    py.image.save_as(fig, filename=figname)
    # py.plot(fig, filename=figname) # Opens in web
else:
    fig = go.Figure(data=data_z3)
    iplot(fig)

In [12]:
ts = ['accVOL_z0_m3', 'accVOL_z1_m3', 'accVOL_z2_m3', 'accVOL_z3_m3', 'accVOL_Bsmt_m3']
#ts = ['accVOL_z3_m3', 'accVOL_Bsmt_m3']
data_vol = []
for i in range(len(ts)):  # Append simulated components
    filename = "resW_" + ts[i] + ".tss"
    sim_pd = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', 'sim'],
                            header=None)
    
    sim = sim_pd.sim.tolist()
    trace = go.Scatter(
        x = x,
        y = sim,
        mode = 'lines',
        name = ts[i],
        line = dict(
            color = palette[i],
            width = 3,
            dash = 'dot')
    )
    data_vol.append(trace)

figname = "Volm3Layers.png"
%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data_vol)
    py.image.save_as(fig, filename=figname)
    # py.plot(fig, filename=figname) # Opens in web
else:
    fig = go.Figure(data=data_vol)
    iplot(fig)

## Lateral flow & retention time

## Nash

In [13]:
start = 1
n_tests = 1
for i in range(start, n_tests+1):
    folder = str(i)
    col = 'Nash.c' + str(i)
    nash = pd.read_table(sim_path + folder + back + "resNash_q_m3.tss", skiprows=4, delim_whitespace=True,
                         names=['Jdays', col],
                         header=None)
    if i == 1:
        nash_all = nash
    else:
        nash_all = nash_all.merge(nash, left_on='Jdays', right_on='Jdays', how='outer')

if short_sim:
    ndata = nash_all[11:] # [10:] (v1)
else:
     ndata = nash_all[176:] 

In [14]:
ndata.head()

,Jdays,Nash.c1
11,177,0.994607
12,178,0.988154
13,179,0.986269
14,180,0.985896
15,181,0.985475


In [15]:
nash = ndata['Nash.c1'].tolist()
x = ndata['Jdays'].tolist()

In [16]:
traceN = go.Scatter(
    x = x,
    y = nash,
    mode = 'lines',
    name = 'Nash.',
    line = dict(
        color = '#F44336',
        width = 3,
        dash = 'dot')
)
n = [traceN]
nfigname = "Nash.png"

%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=n)#, layout=layout)
    py.image.save_as(fig, filename=nfigname)
    # py.plot(fig, filename=figname) # Opens in web
else:
    fig = go.Figure(data=n)#, layout=layout)
    iplot(fig)